In [1]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.0 MB/s eta 0:00:00


In [ ]:
from torch.utils.data import Dataset, Dataloader
from torch_geometric.data import Data


In [ ]:

class WormUpExamplesDataset(Dataset):

  def __init__(self, graphs: list[Data], actions: list[int], rewards: list[int]):
    self.data = zip(graphs, actions, rewards)


